In [ ]:
!pip install praw
import nltk
nltk.download('stopwords')
import warnings

warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

import logging
logging.getLogger('praw').setLevel(logging.ERROR)

In [ ]:
import praw
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from collections import Counter
from nltk.util import bigrams
import string


reddit = praw.Reddit(
    client_id="uTp5HipGLhBhPv1S7oxk8A",
    client_secret="AcT-azqNdIT3onTltclkX4Q3ISUDDQ",
    user_agent="mads",
)

subreddit = reddit.subreddit('netflix')
top_posts = subreddit.top('month', limit=100)

stop_words = set(stopwords.words('english'))
custom_stopwords = {"netflix", "season", "new", "shows", "series", "show", "movie", "movies","streaming","like"}
stop_words = stop_words.union(custom_stopwords)

all_texts = []
for post in top_posts:
    all_texts.append(post.title)
    for comment in post.comments:
        if isinstance(comment, praw.models.MoreComments):
            continue
        all_texts.append(comment.body)
all_texts = " ".join(all_texts).lower()

tokens = [word.strip(string.punctuation) for word in all_texts.split()]
filtered_tokens = [word for word in tokens if word not in stop_words and len(word) > 1]

bi_grams = list(bigrams(filtered_tokens))

bi_gram_freq = Counter(bi_grams)


top_bi_grams = bi_gram_freq.most_common(20)
bi_gram_strings, bi_gram_counts = zip(*[(f"{word1} {word2}", count) for (word1, word2), count in top_bi_grams])

plt.figure(figsize=(10,7))
plt.barh(bi_gram_strings, bi_gram_counts, color='skyblue')
plt.xlabel('Counts')
plt.title('Top bi-grams in Netflix Subreddit Past Month')
plt.gca().invert_yaxis()
plt.show()


In [ ]:
import praw
import matplotlib.pyplot as plt
import datetime
from collections import defaultdict
import logging

logging.getLogger('praw').setLevel(logging.ERROR)

reddit = praw.Reddit(
    client_id="uTp5HipGLhBhPv1S7oxk8A",
    client_secret="AcT-azqNdIT3onTltclkX4Q3ISUDDQ",
    user_agent="mads",
)

subreddit = reddit.subreddit('netflix')
top_posts = subreddit.top('month', limit=None)

daily_posts = defaultdict(int)
daily_comments = defaultdict(int)

for post in top_posts:
    day = datetime.datetime.utcfromtimestamp(post.created_utc).strftime('%Y-%m-%d')
    daily_posts[day] += 1
    daily_comments[day] += post.num_comments

dates = sorted(daily_posts.keys())
post_counts = [daily_posts[date] for date in dates]
comment_counts = [daily_comments[date] for date in dates]

fig, ax1 = plt.subplots(figsize=(10, 7))

color = 'tab:blue'
ax1.set_xlabel('Date')
ax1.set_ylabel('Number of Posts', color=color)
ax1.bar(dates, post_counts, color=color)
ax1.tick_params(axis='y', labelcolor=color)
ax1.set_xticks(dates)
ax1.set_xticklabels(dates, rotation=45, ha='right')

ax2 = ax1.twinx()  
color = 'tab:red'
ax2.set_ylabel('Number of Comments', color=color)
ax2.plot(dates, comment_counts, color=color, marker='o')
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  
plt.title('Number of Posts and Comments on Netflix Subreddit Over the Last Month')
plt.show()


In [ ]:
import praw
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

reddit = praw.Reddit(
    client_id="uTp5HipGLhBhPv1S7oxk8A",
    client_secret="AcT-azqNdIT3onTltclkX4Q3ISUDDQ",
    user_agent="mads",
)

subreddit = reddit.subreddit('netflix')
top_posts = subreddit.top('month', limit=100)

stop_words = set(STOPWORDS)
custom_stopwords = {"netflix", "season", "new", "shows", "series", "show", "movie", "movies", "streaming", 'like', 'feel', 'felt'}
stop_words = stop_words.union(custom_stopwords)

all_comments = []
for post in top_posts:
    for comment in post.comments:
        if isinstance(comment, praw.models.MoreComments):
            continue
        all_comments.append(comment.body)
all_comments = " ".join(all_comments).lower()

wordcloud = WordCloud(stopwords=stop_words, background_color="white", width=800, height=600).generate(all_comments)

plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title("Word Cloud for Reddit Comments in 'netflix' Subreddit")
plt.show()
